In [ ]:
%matplotlib inline
from pprint import pprint
from matplotlib import pyplot as plt
import itertools
import pandas
import seaborn
import numpy

import market
import simulate
import harvesting
import withdrawal
import metrics
import montecarlo
import mortality

In [ ]:
seaborn.set(style="whitegrid")
seaborn.set_context('poster')

In [ ]:
Market = market.Returns_US_1871()

In [ ]:
def get_rq(stock_pct, age, withdrawal_pct):
    # I can't figure out how to to joint life expectancy so I'll
    # just use female life expectancy for now :/
    life_expectancy = mortality.life_expectancy(None, age)
    mean = montecarlo.simba_mean[stock_pct]
    stddev = montecarlo.simba_stddev[stock_pct]
    
    return metrics.probability_of_ruin(mean, stddev, life_expectancy, float(withdrawal_pct))

In [ ]:
df = pandas.DataFrame(index=numpy.arange(1,26))
for year in range(1871, 2019 - 30 + 1):
    s = simulate.withdrawals(Market.iter_from(year), years=25, harvesting=harvesting.N_60_RebalanceHarvesting, withdraw=withdrawal.ConstantDollar)
    r_series = []
    for (i, age) in zip(s, itertools.count(start=65)):
        r = get_rq(60, age, i.withdraw_n/i.portfolio_pre.value_n)
        r_series.append(r)
    df[year] = r_series
df.head()

In [ ]:
plt.figure(figsize=(11,9))
df.max().plot()

In [ ]:
plt.figure(figsize=(11,9))
df[1960].plot()